<a href="https://colab.research.google.com/github/jiwonleelee/Deepfake-Detection-v2/blob/main/notebooks/01_preprocessing/FF%2B%2B_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install insightface onnxruntime-gpu

In [ ]:
import cv2
import numpy as np
from google.colab import drive
import os
import json
import urllib.request
import tempfile
import random
from tqdm import tqdm
from insightface.app import FaceAnalysis

import shutil
import zipfile

In [ ]:
drive.mount('/content/drive')

In [ ]:
import sys
# 2. ff_utils.py가 저장된 '폴더' 경로를 시스템 경로에 추가
# 주의: 파일명이 아니라 파일이 들어있는 '폴더'까지만 적어주세요.
MODULE_PATH = "/content/drive/MyDrive/Deepfake-Detection-v2/src"
if MODULE_PATH not in sys.path:
    sys.path.append(MODULE_PATH)

# 3. 이제 임포트가 가능합니다
import pp_utils
import importlib
importlib.reload(pp_utils) # 코드를 수정하고 다시 저장했을 때 반영하기 위함

<module 'pp_utils' from '/content/drive/MyDrive/Deepfake-Detection-v2/src/pp_utils.py'>

In [ ]:
# --- 설정 수정 ---
LOCAL_BASE_PATH = "/content/FF_frames_local"  # 코랩 로컬 경로에서 작업
GDRIVE_SAVE_BASE = "/content/drive/MyDrive/Deepfake-Detection-v2/Datasets/FF++" # 결과물이 저장될 구드 폴더
GDRIVE_ZIP_PATH = os.path.join(GDRIVE_SAVE_BASE, "FF_frames.zip")
JSON_DB_PATH = os.path.join(GDRIVE_SAVE_BASE, "ff_face_metadata.json")
VERIFY_SAVE_PATH = os.path.join(GDRIVE_SAVE_BASE, "FF_Verification")

# 구드에 저장
FINAL_SAVE_PATH = "/content/drive/MyDrive/Deepfake-Detection-v2/Datasets/FF++/Images"

NUM_FRAMES = 15
SERVER_URL = "http://kaldir.vc.in.tum.de/faceforensics/v3/"
FILELIST_URL = SERVER_URL + "misc/filelist.json"
MANIPULATIONS = ['Deepfakes', 'Face2Face', 'FaceSwap', 'NeuralTextures']
COMPRESSION = 'c23'

In [ ]:
# RetinaFace 초기화 (랜드마크 추출을 위해 landmark2d 추가)
detector = FaceAnalysis(allowed_modules=['detection', 'landmark_2d'], providers=['CUDAExecutionProvider'])
detector.prepare(ctx_id=0, det_size=(640, 640))

def resize_with_letterbox(image, target_size=(512, 512)):
    if image.size == 0: return np.zeros((target_size[1], target_size[0], 3), dtype=np.uint8)
    h, w = image.shape[:2]
    scale = min(target_size[0]/w, target_size[1]/h)
    nw, nh = int(w * scale), int(h * scale)

    img_resized = cv2.resize(image, (nw, nh))
    canvas = np.zeros((target_size[1], target_size[0], 3), dtype=np.uint8)
    dx, dy = (target_size[0]-nw)//2, (target_size[1]-nh)//2
    canvas[dy:dy+nh, dx:dx+nw] = img_resized
    return canvas

In [ ]:
def process_and_extract_combined(video_names, sub_path, category_name, label, detector, is_real=True):
    # 구글 드라이브 최종 저장 경로 설정
    save_dir = os.path.join(FINAL_SAVE_PATH, category_name)
    os.makedirs(save_dir, exist_ok=True)

    # JSON DB 로드 (기존 데이터가 있으면 불러와서 이어서 작업)
    metadata = {}
    if os.path.exists(JSON_DB_PATH):
        with open(JSON_DB_PATH, 'r') as f:
            metadata = json.load(f)

    for v_name in tqdm(video_names, desc=f"Processing {category_name}"):
        video_id = v_name.split('.')[0]
        video_folder = os.path.join(save_dir, video_id)

        # 🔥 [이어하기 로직] 이미 폴더가 있고 프레임이 다 차있으면 스킵
        if os.path.exists(video_folder) and len(os.listdir(video_folder)) >= NUM_FRAMES:
            continue

        os.makedirs(video_folder, exist_ok=True)
        video_url = SERVER_URL + sub_path + v_name
        target_id = video_id.split('_')[0] if not is_real else video_id

        with tempfile.NamedTemporaryFile(suffix='.mp4') as temp_v:
            try:
                urllib.request.urlretrieve(video_url, temp_v.name)
                cap = cv2.VideoCapture(temp_v.name)
                total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                if total <= 0: continue
                interval = max(1, total // NUM_FRAMES)

                last_face_info = None
                video_faces_found = False
                frames_cache = []
                face_crops_temp = []
                current_video_metadata = [None] * NUM_FRAMES

                for i in range(NUM_FRAMES):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, i * interval)
                    ret, frame = cap.read()
                    if not ret or frame is None:
                        face_crops_temp.append(None)
                        continue

                    frames_cache.append(frame)
                    face_to_save = None
                    used_info = None

                    if is_real:
                        # [REAL] GPU 가속 얼굴 검출
                        faces = detector.get(frame)
                        if faces:
                            used_info = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0]) * (x.bbox[3]-x.bbox[1]))
                            face_to_save = pp_utils.get_aligned_face(frame, used_info)

                            if face_to_save is not None:
                                # 소급 적용
                                if not video_faces_found and i > 0:
                                    for prev_idx in range(len(face_crops_temp)):
                                        if face_crops_temp[prev_idx] is None:
                                            retro_crop = pp_utils.get_aligned_face(frames_cache[prev_idx], used_info)
                                            face_crops_temp[prev_idx] = retro_crop
                                            current_video_metadata[prev_idx] = {
                                                "bbox": used_info.bbox.tolist(),
                                                "kps": used_info.kps.tolist() if hasattr(used_info, 'kps') else None
                                            }
                                last_face_info = used_info
                                video_faces_found = True

                        if face_to_save is None and last_face_info is not None:
                            face_to_save = pp_utils.get_aligned_face(frame, last_face_info)
                            used_info = last_face_info
                    else:
                        # [FAKE] JSON 좌표 재활용 (구드에서 불러온 metadata 사용)
                        if target_id in metadata and i < len(metadata[target_id]):
                            info = metadata[target_id][i]
                            if info:
                                class FaceInfo: pass
                                used_info = FaceInfo()
                                used_info.bbox = np.array(info['bbox'])
                                used_info.kps = np.array(info['kps']) if info['kps'] else None
                                face_to_save = pp_utils.get_aligned_face(frame, used_info)
                                video_faces_found = True

                    face_crops_temp.append(face_to_save)
                    if is_real and used_info and face_to_save is not None:
                        current_video_metadata[i] = {
                            "bbox": used_info.bbox.tolist(),
                            "kps": used_info.kps.tolist() if hasattr(used_info, 'kps') else None
                        }

                # 파일 저장 (구글 드라이브로 직접 쓰기)
                if video_faces_found:
                    for idx, crop in enumerate(face_crops_temp):
                        if crop is not None:
                            cv2.imwrite(os.path.join(video_folder, f"f{idx:03d}_{label}.jpg"), crop)
                else:
                  # 얼굴 인식 실패 시 전체 이미지 저장하지 않고 폴더 삭제 후 스킵
                  print(f"⚠️ {video_id}: 얼굴 인식 실패로 제외합니다.")
                  if os.path.exists(video_folder):
                    shutil.rmtree(video_folder)

                # 🔥 [중요] 매 영상 완료 시마다 JSON 실시간 업데이트
                if is_real:
                    metadata[video_id] = current_video_metadata
                    with open(JSON_DB_PATH, 'w') as f:
                        json.dump(metadata, f)

                cap.release()
            except Exception as e:
                print(f"❌ Error {v_name}: {e}")

In [ ]:
# --- 샘플 확인 설정 ---
def run_visual_verification(detector):
    """
    각 조작 방식별로 첫 번째 Fake 영상을 선정하고,
    그에 대응하는 Real 영상을 자동으로 찾아 검증 리스트를 구성합니다.
    """
    print("🔍 샘플 검증 프로세스를 시작합니다...")
    if os.path.exists(VERIFY_SAVE_PATH):
        shutil.rmtree(VERIFY_SAVE_PATH)
    os.makedirs(VERIFY_SAVE_PATH, exist_ok=True)

    # 1. 데이터 리스트 가져오기
    real_videos, fake_videos_dict = pp_utils.get_id_matched_dataset()
    verify_list = []
    temp_metadata = {}

    # 2. 각 카테고리별로 Fake 샘플 하나씩 선정하고, 대응하는 Real도 세트로 추가
    # 이렇게 해야 Fake 처리 시 필요한 Real의 JSON 좌표가 반드시 존재하게 됩니다.
    for method in MANIPULATIONS:
        v_list = fake_videos_dict.get(method, [])
        if not v_list:
            print(f"⚠️ 경고: {method} 카테고리에 데이터가 없습니다.")
            continue

        # 첫 번째 가짜 영상 선택 (예: '751_752.mp4')
        fake_v_name = v_list[0]
        target_id = fake_v_name.split('_')[0] # '751' 추출

        # 해당 가짜 영상의 원본(Real) 파일명 생성 (예: '751.mp4')
        real_v_name = target_id + ".mp4"

        if real_v_name in real_videos:
            # Real 먼저 추가 (좌표 생성을 위해)
            verify_list.append(("Real", [real_v_name], f"original_sequences/youtube/{COMPRESSION}/videos/", 0, True))
            # 그 다음 Fake 추가 (좌표 재활용을 위해)
            verify_list.append((f"Fake/{method}", [fake_v_name], f"manipulated_sequences/{method}/{COMPRESSION}/videos/", 1, False))
        else:
            print(f"⚠️ 경고: {method}의 타겟 ID {target_id}에 해당하는 원본 영상을 찾을 수 없습니다.")

    # 3. 위에서 구성된 리스트 순차 처리
    for category, v_names, sub_path, label, is_real in verify_list:
        v_name = v_names[0]
        video_id = v_name.split('.')[0]
        target_id = video_id.split('_')[0] if not is_real else video_id

        # 영상별 개별 폴더 생성
        video_folder = os.path.join(VERIFY_SAVE_PATH, category, video_id)
        os.makedirs(video_folder, exist_ok=True)

        print(f"📸 {category} 처리 중: {video_id}")
        video_url = SERVER_URL + sub_path + v_name

        with tempfile.NamedTemporaryFile(suffix='.mp4') as temp_v:
            try:
                urllib.request.urlretrieve(video_url, temp_v.name)
                cap = cv2.VideoCapture(temp_v.name)
                for i in range(5): # 5프레임씩 검증
                    cap.set(cv2.CAP_PROP_POS_FRAMES, i * 10)
                    ret, frame = cap.read()
                    if not ret: break

                    face_to_save = None
                    if is_real:
                        # [REAL] 직접 검출 및 좌표 저장
                        faces = detector.get(frame)
                        if faces:
                            # score가 None인 경우를 대비해 0.0 처리
                            used_info = sorted(faces, key=lambda x: (getattr(x, 'score', 0.0) or 0.0), reverse=True)[0]
                            face_to_save = pp_utils.get_aligned_face(frame, used_info)
                            if target_id not in temp_metadata: temp_metadata[target_id] = []
                            temp_metadata[target_id].append({
                                "bbox": used_info.bbox.tolist(),
                                "kps": used_info.kps.tolist() if hasattr(used_info, 'kps') else None
                            })
                    else:
                        # [FAKE] 저장된 좌표 재활용
                        if target_id in temp_metadata and i < len(temp_metadata[target_id]):
                            class FaceInfo: pass
                            info = temp_metadata[target_id][i]
                            f_obj = FaceInfo()
                            f_obj.bbox = np.array(info['bbox'])
                            f_obj.kps = np.array(info['kps']) if info['kps'] else None
                            face_to_save = pp_utils.get_aligned_face(frame, f_obj)

                    if face_to_save is not None:
                        cv2.imwrite(os.path.join(video_folder, f"f{i:03d}_{label}.jpg"), face_to_save)
                cap.release()
            except Exception as e:
                print(f"   -> ❌ {video_id} 에러: {e}")

    print(f"\n✅ 검증 완료! 구글 드라이브 폴더를 확인하세요: {VERIFY_SAVE_PATH}")

In [ ]:
# 1. 환경 준비 (폴더 삭제 로직 제거 - 이어하기를 위해)
if not os.path.exists(FINAL_SAVE_PATH):
    os.makedirs(FINAL_SAVE_PATH, exist_ok=True)
    print(f"✅ 새 저장 디렉토리 생성: {FINAL_SAVE_PATH}")
else:
    print(f"ℹ️ 기존 디렉토리를 유지합니다. (이어하기 모드)")

# 2. 데이터 리스트 준비
real_videos, fake_videos_dict = pp_utils.get_id_matched_dataset(FILELIST_URL, MANIPULATIONS)

# 3. 본 작업 시작
print("\n🎬 [1/2] Real 영상 처리 및 JSON 좌표 DB 구축 시작...")
real_sub_path = f"original_sequences/youtube/{COMPRESSION}/videos/"
process_and_extract_combined(real_videos, real_sub_path, "Real", 0, detector, is_real=True)

print("\n🎬 [2/2] Fake 영상 ID 매칭 처리 시작 (좌표 재활용)...")
for method, v_list in fake_videos_dict.items():
    fake_sub_path = f"manipulated_sequences/{method}/{COMPRESSION}/videos/"
    process_and_extract_combined(v_list, fake_sub_path, f"Fake/{method}", 1, detector, is_real=False)

print("\n🎉 모든 전처리가 완료되었습니다. 구글 드라이브를 확인하세요.")

ℹ️ 기존 디렉토리를 유지합니다. (이어하기 모드)

🎬 [1/2] Real 영상 처리 및 JSON 좌표 DB 구축 시작...


Processing Real: 100%|██████████| 1000/1000 [1:59:51<00:00,  7.19s/it]



🎬 [2/2] Fake 영상 ID 매칭 처리 시작 (좌표 재활용)...


Processing Fake/NeuralTextures: 100%|██████████| 250/250 [28:48<00:00,  6.92s/it]


🎉 모든 전처리가 완료되었습니다. 구글 드라이브를 확인하세요.
